In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/notebooks/")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [ ]:
directory = data_folder+"clean_data/agricoltura/"
for file in os.listdir(directory):
    print(file)
    display(pd.read_csv(directory+file).head())

---

# Small fix 

In [ ]:
result = pd.read_csv(directory+"grazing_census_2020.csv").pivot_table(index='REF_AREA', columns='TYPE_OF_HERD', values='OBS_VALUE')
(result == 0).sum(axis=1).sum() # hence all the NaN are 0s

In [ ]:
save_clean_data(result.fillna(0), "grazing_census_2020", directory)

---

In [ ]:
result2 = pd.read_csv(directory+"agricultural_land_usage_census_2020.csv").pivot_table(index='REF_AREA', columns='DATA_TYPE', values='OBS_VALUE')
(result2 == 0).sum(axis=1).sum() # some 0s are not NaN

In [ ]:
save_clean_data(result2.fillna(0), "agricultural_land_usage_census_2020", directory)

---

In [ ]:
pd.read_csv(directory+"farming_census_2020.csv")
result3 = pd.read_csv(directory+"farming_census_2020.csv").pivot_table(index='REF_AREA', columns='TYPE_OF_CROP', values='OBS_VALUE')
(result3 == 0).sum(axis=1).sum() # all NaN are 0s

In [ ]:
save_clean_data(result3.fillna(0), "farming_census_2020", directory)

---

# Geomerge

In [ ]:
confini = gpd.read_file(data_folder+"clean_data/confini_piemonte/confini_piemonte.shp")
for file in ['farming_census_2020.csv', 'agricultural_land_usage_census_2020.csv', 'grazing_census_2020.csv']:
    print(file)
    save_clean_data(gpd.GeoDataFrame(
        pd.read_csv(directory+file).merge(confini[['Codice Com', 'geometry']].rename(columns={'Codice Com': 'REF_AREA'}), on = 'REF_AREA', how='inner')
        ).reset_index(drop=True), file[:-4])
    
for file in ['grazing_census_2010.csv', 'farming_land_usage_census_2010.csv']:
    print(file)
    save_clean_data(gpd.GeoDataFrame(
        pd.read_csv(directory+file).merge(confini[['Codice Com', 'geometry']].rename(columns={'Codice Com': 'Codice Comune'}), on = 'Codice Comune', how='inner')
        ).reset_index(drop=True), file[:-4])

In [ ]:
save_clean_data(gpd.GeoDataFrame(
        pd.read_csv(directory+'agricultural_land_usage_census_2020.csv').merge(confini[['Codice Com', 'geometry']].rename(columns={'Codice Com': 'REF_AREA'}), on = 'REF_AREA', how='inner')
        ).reset_index(drop=True), 'agricultural_land_usage_census_2020')

---